## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-22-01-15-56 +0000,nypost,NTSB investigating after 4 killed in fiery SoC...,https://nypost.com/2026/02/21/us-news/ntsb-inv...
1,2026-02-22-01-10-55 +0000,nypost,Creepy men spotted lurking in influencer’s bac...,https://nypost.com/2026/02/21/us-news/creepy-m...
2,2026-02-22-01-07-02 +0000,nyt,Blizzard Warnings Issued for Swath of East Coa...,https://www.nytimes.com/2026/02/21/us/blizzard...
3,2026-02-22-01-00-01 +0000,nyt,Traveling This Weekend? Here’s What to Know Ab...,https://www.nytimes.com/2026/02/21/travel/flig...
4,2026-02-22-01-00-00 +0000,wsj,The Biggest New Fans of 401(k)s Are Small Busi...,https://www.wsj.com/personal-finance/retiremen...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2496/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,46
144,court,22
143,supreme,19
37,tariffs,17
178,tariff,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
97,2026-02-21-16-56-26 +0000,cbc,Trump says he's raising new global tariff rate...,https://www.cbc.ca/news/politics/trump-global-...,145
89,2026-02-21-17-23-00 +0000,wsj,President Trump said he would increase to 15% ...,https://www.wsj.com/politics/policy/trump-incr...,144
155,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...,144
99,2026-02-21-16-46-07 +0000,wapo,Trump’s response to Supreme Court tariff rulin...,https://www.washingtonpost.com/business/2026/0...,136
138,2026-02-21-11-00-11 +0000,nyt,Supreme Court Ruling on Trump’s Tariffs Change...,https://www.nytimes.com/2026/02/21/world/canad...,129


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
97,145,2026-02-21-16-56-26 +0000,cbc,Trump says he's raising new global tariff rate...,https://www.cbc.ca/news/politics/trump-global-...
155,57,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...
43,35,2026-02-21-21-42-40 +0000,nypost,Ski guides killed in Lake Tahoe avalanche iden...,https://nypost.com/2026/02/21/us-news/blackbir...
36,31,2026-02-21-22-09-19 +0000,bbc,Iran students stage first large anti-governmen...,https://www.bbc.com/news/articles/c5yj2kzkrj0o...
14,29,2026-02-22-00-15-48 +0000,nypost,"Senior MP demands UK probe ex-Prince Andrew, P...",https://nypost.com/2026/02/21/world-news/senio...
49,26,2026-02-21-21-18-52 +0000,nypost,Panicked Dems fear ‘zombie campaigns’ will se...,https://nypost.com/2026/02/21/us-news/californ...
52,25,2026-02-21-20-54-20 +0000,nypost,Israeli strikes on Hezbollah kill 8 as terror ...,https://nypost.com/2026/02/21/world-news/israe...
27,25,2026-02-21-23-36-55 +0000,nypost,Radical activist who likened CPS to genocide m...,https://nypost.com/2026/02/21/us-news/radical-...
145,24,2026-02-21-11-00-00 +0000,latimes,LAUSD borrowing $250 million to settle sex abu...,https://www.latimes.com/california/story/2026-...
201,23,2026-02-21-02-00-00 +0000,wsj,"James Talarico, running for the U.S. Senate in...",https://www.wsj.com/politics/elections/james-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
